In [30]:
import tensorflow as tf
import utils
import models
import embedding_only
import os
import word2vec as w2v
import numpy as np

In [55]:
stuffs = utils.load_run_data(run_id='conv/default_conv')

/Users/raymond/Playground/WhatsTheWord
/Users/raymond/Playground/WhatsTheWord/logs/conv/default_conv
Using run: 20180506-2003


In [56]:
run_config, dictionary, reversed_dictionary, full_folderpath = stuffs

In [57]:
run_config

{'batch_size': 256,
 'embedding_size': 128,
 'num_sampled': 64,
 'num_skips': 1,
 'num_steps': 50000,
 'skip_window': 4,
 'vocabulary_size': 5000}

In [53]:
tf.reset_default_graph()
graph = tf.Graph()
session = tf.Session(graph=graph)
with graph.as_default():
    tensors = embedding_only._load_tensors(run_config['vocabulary_size'], run_config['embedding_size'], graph)
    embeddings = tensors['embeddings']
    reversed_dictionary[0]
    test_words = "some days i wonder if you still think about"
    split_words = test_words.split()
    words = split_words[:4] + split_words[5:]
    print("example: {}".format(words))
    vectorized = [[dictionary.get(w + '\n', 0) for w in words]]
    print('vectorized: {}'.format(vectorized))
#     one_hot = tf.one_hot(vectorized, run_config['vocabulary_size'])
    embedding_input = tf.nn.embedding_lookup(embeddings, vectorized)
    embedding_placeholder = tf.placeholder(tf.float32, shape=[1, 8, 128])
    with tf.name_scope("model"):
        m = models.CNNModel(embedding_placeholder, **run_config)
        print(embedding_input.shape)
        out_tensor = m.load_tesnsors()
        softmax = tf.nn.softmax(out_tensor)
    with session as sess:
        saver = tf.train.Saver()
        saver.restore(sess, os.path.join(full_folderpath, 'model.ckpt'))
        input_array = sess.run(embedding_input)
        results = sess.run(softmax, feed_dict={embedding_placeholder: input_array})
        """??? What's up next ??? Pass in the embedding input sess.run() and get the output"""
# placeholder_embed = tf.nn.embedding_lookup(embeddings, placeholder_input)

example: ['some', 'days', 'i', 'wonder', 'you', 'still', 'think', 'about']
vectorized: [[83, 417, 1, 654, 3, 109, 117, 92]]
(1, 8, 128)
Tensor("Placeholder:0", shape=(1, 8, 128), dtype=float32)
<tf.Variable 'model/conv_weights/Variable:0' shape=(2, 128, 64) dtype=float32_ref>
<tf.Variable 'model/conv_weights/Variable_1:0' shape=(512, 2500) dtype=float32_ref>
Tensor("model/conv_weights/conv1d/Squeeze:0", shape=(1, 8, 64), dtype=float32)
<tf.Variable 'model/conv_biases/Variable:0' shape=(8, 64) dtype=float32_ref>
Tensor("model/calulations/Flatten/Reshape:0", shape=(1, 512), dtype=float32)
<tf.Variable 'model/conv_weights/Variable_1:0' shape=(512, 2500) dtype=float32_ref>
<tf.Variable 'model/conv_biases/Variable_1:0' shape=(2500,) dtype=float32_ref>
Tensor("model/calulations/add_1:0", shape=(1, 2500), dtype=float32)
INFO:tensorflow:Restoring parameters from /Users/raymond/Playground/WhatsTheWord/logs/conv/testing_conv/20180614-2304/model.ckpt


In [54]:
sum(results[0])
results[0].shape
idx = np.argmax(results[0])
print("predicted word: {}".format(reversed_dictionary[idx]))
truth = dictionary['what\n']
print("expected {} got {}".format(truth, idx))
print("softmax vals: \n\texpected word '{}': {}\n\tpredicted word '{}': {}".format(truth, results[0][truth], idx, results[0][idx]))

predicted word: i

expected 33 got 1
softmax vals: 
	expected word '33': 0.0
	predicted word '1': 1.0


In [25]:
s = [0,1,2,3,4,5,6,7,8]
ex = w2v._make_model_example(s, 9, True)

In [61]:
combined = np.array([results[0], results[0]])
combined.shape
tf.argmax(combined, axis=1)

<tf.Tensor 'ArgMax_2:0' shape=(2,) dtype=int64>

In [63]:
tf.cast(1, tf.int64)

<tf.Tensor 'Cast:0' shape=() dtype=int64>